## Personas que deberías conocer

El vicepresindente de "fraternización" quiere animar a que hallan más conexiones entre los miembros de la red y te pide que diseñes un "suggester".

El primer instinto es sugerir a los amigos de los amigos. Estos son fáciles de calcular: para cada uno de los amigos del usuario, iterar sobre los amigos de esa persona, y recoger todos los resultados

In [2]:
# Usuarios

users = [
    {"id": 0, "name": "Hero"},
    {"id": 1, "name": "Dunn"},
    {"id": 2, "name": "Sue"},
    {"id": 3, "name": "Chi"},
    {"id": 4, "name": "Thor"},
    {"id": 5, "name": "Clive"},
    {"id": 6, "name": "Hicks"},
    {"id": 7, "name": "Devin"},
    {"id": 8, "name": "Kate"},
    {"id": 9, "name": "Klein"},
]

In [3]:
# Friendships

friendships = [(0, 1), (0, 2), (1, 2), (1, 3), (2, 3), (3, 4),
               (4, 5), (5, 6), (5, 7), (6, 8), (7, 8), (8, 9)]

In [4]:
for user in users:        # para cada user en users
    user["friends"] = []  # crear propiedad friends como una lista vacía

In [5]:
for i, j in friendships:
    # users[i] es el usuario que su id es i
    users[i]["friends"].append(users[j]) # agregar j como amigo de i
    users[j]["friends"].append(users[i])  # agregar i como amigo de j

In [6]:
def friend_of_friends(user):
    # foaf es el abreviado de "friend of a friend"
    return [
        foaf["id"]
        for friend in user["friends"] # para cada amigo de user
        for foaf in friend["friends"] # obtener sus amigos
    ]

Con este código estamos obteniendo los id de los amigos de los amigos de cada usuario, una vez más se emplea la "manera declarativa" de crear una lista en Python. (Tuve que copiar todo el código de la lista de usuarios y la creacion de la lista de amigos para que funcionara correctamente el código nuevo, el objetivo de estos Notebooks es que sean eso Notebooks donde se puede correr el código y que funcione)

In [7]:
friend_of_friends(users[0])

[0, 2, 3, 0, 1, 3]

Hemos llamado a la función que hemos creado para saber quienes son los amigos de los amigos de Hero (id 0) y nos ha devuelto una lista. La cual contiene el 0 dos veces, ya que de hecho Hero es amigo de sus dos amigos. Incluye ademas los usuarios 1 y 2, a pesar de que ya son amigos de Hero e incluye al usuario Chi (id 3) dos veces, porque Chi es accesible a través de dos amigos diferentes

<img align="center" style="padding:10px;" src="img/fig1.1.png">

In [9]:
print([friend["id"] for friend in users[0]["friends"]])
print([friend["id"] for friend in users[1]["friends"]])
print([friend["id"] for friend in users[2]["friends"]])

[1, 2]
[0, 2, 3]
[0, 1, 3]
